<a href="https://colab.research.google.com/github/mahb97/Wake2vec/blob/main/Wake2Vec_tokens_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wake2vec · Token Injection & Training

**Goal.** Augments the tokenizer with a Joyce-heavy lexicon, smart-initialises new embeddings, and trains (embeddings-only, optionally with a tiny LoRA) on Wake-dense windows. I finish with geometry + behavior metrics and a portfolio-ready hero figure.

---

## What this notebook does
1. **Load base model & tokenizer** (TinyLlama on GPU, or `distilgpt2` on CPU for smoke tests).
2. **Inject tokens** from `wake_lexicon.txt`:
   - Adds bare forms and start-of-word variants (`▁token`) for SentencePiece-style tokenization.
   - Ties the output head (`lm_head`) to the input embeddings after every resize.
3. Smart init new vectors as the mean of base subword embeddings + small noise.
4. **Dataset**: build Wake-dense paragraph windows (sliding radius) for focused training.
5. **Train**:
   - Freeze almost everything; **embeddings-only** are trainable.
   - Optionally enable a tiny LoRA on `q/k/v/o` (e.g., `r=8`) to let attention “feel” new tokens.
   - Cosine LR + warmup, gradient clipping, early stopping on validation loss.
6. **Evaluate**:
   - **Geometry**: PIP loss, isotropy, top-k neighbor overlap (before/after snapshot).
   - **Behavior**: nearest-neighbor tables + Joyce-style completions.
   - Save `results/metrics_summary.csv` and `results/hero.png`.

---

## Inputs
- `data/wake_lexicon.txt` one token per line (from Notebook 00).
- `data/finnegans_wake.txt` UTF-8 text used to build Wake-dense windows.

## Outputs
- `results/metrics_summary.csv` PIP loss, isotropy, top-10 overlap.
- `results/hero.png` UMAP of neighborhoods + three sample completions.
- `Wake2vec_adapter/minipack/` minimal adapter (rows of the input embedding for new tokens).

---

## Controls (tweak for experiments)
- **Base model**: `BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"` (or `"distilgpt2"` on CPU).
- **LoRA**: `USE_LORA = True` / False; typical `r=8, alpha=16, dropout=0.05`.
- **Windows radius**: `RADIUS = 2` paragraphs (increase to 3 for more context).
- **Training**: `epochs=3`, `learning_rate≈6e-4`, `grad_accum=8`, cosine + 10% warmup, `max_grad_norm=1.0`.
- **Tokenization**: `max_length=512` (lower to 384 if VRAM is tight).

---

## Repro & VRAM tips (T4)
- During training I set:
  - `model.config.use_cache = False`
  - `model.gradient_checkpointing_enable()`
  - `fp16=True` (T4 prefers fp16; re-enable cache only for generation)
- If eval spikes VRAM: `eval_accumulation_steps=16` and/or `max_length=384`.
- After any resize or embedding edits, I retie:  
  `model.get_output_embeddings().weight = model.get_input_embeddings().weight`.

---

## Baseline → After metrics
- I snapshot a baseline before training (probe IDs + pairwise inner-product matrix).
- After training I compute:
  - **PIP loss** = `||P0 − P1||_F / ||P0||_F`
  - **Isotropy** = geomean/mean of covariance eigenvalues
  - **Top-10 overlap** of nearest neighbors
- I append results to `results/metrics_summary.csv`.

---

## Demo sampling (consistent settings)
- `temperature=1.15`, `top_p=0.95`, `repetition_penalty=1.1`, `no_repeat_ngram_size=3`
- I save three fixed-prompt completions into the hero figure for easy comparison.

---

## Optional bonus (continual pretraining)
If I want a neutral-English refresh with optional Joyce style, I run `02b_continual_pretrain_gutenmix.ipynb`:
- Mix a small diverse Gutenberg corpus (≈95–98%) with Wake windows (≈2–5%).
- Add a `<STYLE=Wake>` control token.
- Use a KL-to-teacher guard on general English batches to prevent drift.

---

## Checklist for a clean run
- [ ] Tokens + `▁` variants added; head tied after resize  
- [ ] Smart init applied  
- [ ] Wake-dense windows built (radius set)  
- [ ] Embeddings (± LoRA) are the only trainable params  
- [ ] Cosine LR + warmup + clip enabled; early stopping on  
- [ ] Baseline snapshot saved **before** training  
- [ ] Metrics + hero saved **after** training




In [ ]:
from google.colab import drive; drive.mount('/content/drive')
from datetime import datetime; from pathlib import Path
RUN_ID = datetime.now().strftime("wake2vec_%Y%m%d_%H%M")
BASE = Path(f"/content/drive/MyDrive/Wake2vec_runs/{RUN_ID}")
(BASE/'adapter').mkdir(parents=True, exist_ok=True)
(BASE/'results').mkdir(parents=True, exist_ok=True)
print("Saving to", BASE)

In [1]:
!pip -q install transformers==4.43.3 accelerate peft==0.11.1 datasets umap-learn faiss-cpu matplotlib==3.8.4 sentencepiece

import os, re, math, random, json, unicodedata
from pathlib import Path
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# distilgpt2 for CPU smoke tests
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.3 MB/s eta 0:00:00
Device: cuda


In [2]:
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=DTYPE, device_map="auto")
print("Base vocab size:", tok.vocab_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Base vocab size: 32000


In [3]:
# wake_lexicon.txt
lex_path = None
try:
    from google.colab import files
    uploaded = files.upload()
    lex_path = list(uploaded.keys())[0]
except Exception:
    lex_path = "/content/wake_lexicon.txt"

raw = []
p = Path(lex_path)
if p.exists():
    raw = p.read_text(encoding="utf-8", errors="ignore").splitlines()
else:
    print("WARNING: wake_lexicon.txt not found; provide a path or upload.")

seen = set(); new_terms = []
existing = tok.get_vocab()
for w in raw:
    t = unicodedata.normalize("NFC", w.strip())
    if t and (t not in seen) and (t not in existing):
        seen.add(t); new_terms.append(t)

print(f"Loaded {len(new_terms)} NEW tokens.")
print('Sample:', new_terms[:20])

Saving wake_lexicon-3.txt to wake_lexicon-3.txt
Loaded 44500 NEW tokens.
Sample: ['paùpulation', 'générations', 'introdùce', 'fainéants', 'tricarême', 'deathfête', 'grandmère', 'primamère', 'brofèsor', "andrée's", 'pailleté', 'ténèbres', "angèle's", 'choquées', 'traversé', 'arrivées', 'fraîches', 'willingdone', 'earwicker', 'lipoleums']


In [4]:
added = tok.add_tokens(new_terms, special_tokens=False)
if added != len(new_terms):
    print(f"Note: only {added}/{len(new_terms)} newly added (collisions or tokenizer rules).")
model.resize_token_embeddings(len(tok))
print("New vocab size:", tok.vocab_size)

New vocab size: 32000


In [5]:
# come to clean the mess: add start-of-word variants for SentencePiece-style tokenizers and tie output head
START = "▁"
prefixed = [START + t for t in new_terms if not t.startswith(START)]
prefixed = [t for t in prefixed if t not in tok.get_vocab()]
added2 = tok.add_tokens(prefixed, special_tokens=False)
model.resize_token_embeddings(len(tok))
print(f"Added {added2} start-of-word variants. Vocab: {tok.vocab_size}")

# Tie lm_head to input embeddings so generations reflect updated embeddings
if model.get_output_embeddings() is not None:
    model.get_output_embeddings().weight = model.get_input_embeddings().weight
    print("Tied lm_head to input embeddings.")

Added 44242 start-of-word variants. Vocab: 32000
Tied lm_head to input embeddings.


In [6]:
emb = model.get_input_embeddings()

def init_vec(term: str, noise=0.02):
    ids = tok(term, add_special_tokens=False)["input_ids"]
    if ids:
        base = emb.weight[ids].mean(0, keepdim=True)
    else:
        base = torch.randn_like(emb.weight[0:1])
    return (base + noise * torch.randn_like(base)).squeeze(0)

with torch.no_grad():
    targets = new_terms + [("▁"+t) for t in new_terms if not t.startswith("▁")]
    for t in targets:
        idx = tok.convert_tokens_to_ids(t)
        if idx != tok.unk_token_id:
            emb.weight[idx].copy_(init_vec(t))
print("Initialised bare + ▁ variants.")

Initialised bare + ▁ variants.


In [7]:
# Freeze almost everything, train embeddings (optional tiny LoRA for attention projections)
for p in model.parameters():
    p.requires_grad = False
model.get_input_embeddings().weight.requires_grad_(True)

USE_LORA = True  # if False isolates pure embedding drift
if USE_LORA:
    lora_cfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05,
                          target_modules=["q_proj","k_proj","v_proj","o_proj"],
                          bias="none", task_type="CAUSAL_LM")
    model = get_peft_model(model, lora_cfg)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable params:", trainable)

Trainable params: 2252800


In [8]:
# build Wake-saturated paragraph windows, split into train/val
fw_path = None
try:
    from google.colab import files
    uploaded = files.upload()
    fw_path = list(uploaded.keys())[0]
except Exception:
    fw_path = "/content/finnegans_wake.txt"

fw = Path(fw_path).read_text(encoding="utf-8", errors="ignore")
fw = unicodedata.normalize("NFC", fw).replace("\r\n","\n").replace("\r","\n")

import re, random
paras = [p.strip() for p in re.split(r"\n\s*\n", fw) if p.strip()]
probe = {t.lower() for t in new_terms if len(t) <= 40}

hits = [i for i,p in enumerate(paras) if any(t in p.lower() for t in probe)]
RADIUS = 2
windows = [" ".join(paras[max(0,i-RADIUS):min(len(paras), i+RADIUS+1)]) for i in hits]
windows = list(dict.fromkeys(windows))
random.shuffle(windows)
if len(windows) < 500:
    windows += random.sample(paras, k=min(2000, len(paras)))

split = int(0.9 * len(windows))
train_texts, val_texts = windows[:split], windows[split:]

def tok_fn(batch):
    return tok(batch["text"], truncation=True, max_length=512)

ds = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts}).map(tok_fn, batched=True, remove_columns=["text"]),
    "validation": Dataset.from_dict({"text": val_texts}).map(tok_fn, batched=True, remove_columns=["text"]),
})
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)
print("Train/Val sizes:", len(ds["train"]), len(ds["validation"]))

Saving FinnegansWake.txt to FinnegansWake.txt


Map:   0%|          | 0/1650 [00:00<?, ? examples/s]

Map:   0%|          | 0/184 [00:00<?, ? examples/s]

Train/Val sizes: 1650 184


In [10]:
# BASELINE SNAPSHOT
import os, time, json, numpy as np
from sklearn.metrics.pairwise import cosine_similarity

timestamp = int(time.time())
BASE_DIR = f"/content/Wake2vec_baseline_{timestamp}"
os.makedirs(BASE_DIR, exist_ok=True)

# pick a probe of new tokens + a small control of common anchors
probe_new = new_terms[:200]
anchors   = ["river","history","book","dream","night","language","irish","cork","dublin"]
probe = [t for t in probe_new if tok.convert_tokens_to_ids(t) != tok.unk_token_id][:150] + anchors

ids = [tok.convert_tokens_to_ids(t) for t in probe]
W  = model.get_input_embeddings().weight.detach().cpu().float().numpy() # Convert to float32 before numpy
P0 = W[ids] @ W[ids].T  # pairwise inner products

np.save(f"{BASE_DIR}/probe_ids.npy", np.array(ids, dtype=np.int32))
np.save(f"{BASE_DIR}/P0.npy", P0)
json.dump(probe, open(f"{BASE_DIR}/probe_tokens.json","w"), ensure_ascii=False, indent=2)

# neighbor table (top-8) for a small visible subset
inv = {i:t for t,i in tok.get_vocab().items()}
def neighbors(idx, k=8):
    sims = cosine_similarity(W[idx:idx+1], W)[0]
    top = np.argsort(-sims)[:k]
    return [inv.get(int(j), f"<{j}>") for j in top]

nn0 = {inv[ids[i]]: neighbors(ids[i], 8) for i in range(min(30,len(ids)))}
json.dump(nn0, open(f"{BASE_DIR}/neighbors_before.json","w"), ensure_ascii=False, indent=2)

print("Baseline saved:", BASE_DIR)

Baseline saved: /content/Wake2vec_baseline_1761424464


In [14]:
# metrics
import numpy as np, json
from sklearn.metrics.pairwise import cosine_similarity

# baseline
BASE_DIRS = sorted([p for p in os.listdir("/content") if p.startswith("Wake2vec_baseline_")])
assert BASE_DIRS, "Run the baseline snapshot first."
BASE_DIR = "/content/" + BASE_DIRS[-1]
ids  = np.load(f"{BASE_DIR}/probe_ids.npy")
P0   = np.load(f"{BASE_DIR}/P0.npy")
probe_tokens = json.load(open(f"{BASE_DIR}/probe_tokens.json"))

# current embeddings
W = model.get_input_embeddings().weight.detach().cpu().float().numpy() # Convert to float32 before numpy
P1 = W[ids] @ W[ids].T

# PIP loss (Frobenius)
pip = np.linalg.norm(P0 - P1, 'fro') / (np.linalg.norm(P0, 'fro') + 1e-12)

# isotropy (geomean/mean of eigenvalues of covariance on the probe vectors)
X = W[ids] - W[ids].mean(0, keepdims=True)
C = (X.T @ X) / (len(ids)-1)
evals = np.clip(np.linalg.eigvalsh(C), 1e-12, None)
isotropy = (np.exp(np.log(evals).mean())) / (evals.mean())

# top-k overlap (k=10) before/after
def topk_neighbors(mat, k=10):
    return np.argsort(-mat, axis=1)[:, :k]

N0 = topk_neighbors(P0, k=10)
N1 = topk_neighbors(P1, k=10)
overlap = np.mean([len(set(N0[i]).intersection(set(N1[i]))) / 10.0 for i in range(len(ids))])

import pandas as pd, os
os.makedirs("/content/results", exist_ok=True)
pd.DataFrame([{
    "pip_loss": float(pip),
    "isotropy": float(isotropy),
    "top10_overlap": float(overlap),
    "num_probe_tokens": int(len(ids))
}]).to_csv("/content/results/metrics_summary.csv", index=False)
print("metrics:", {"pip_loss":pip, "isotropy":isotropy, "top10_overlap":overlap})
print("Wrote /content/results/metrics_summary.csv")

metrics: {'pip_loss': np.float32(0.0), 'isotropy': np.float32(1.2080784e-07), 'top10_overlap': np.float64(1.0)}
Wrote /content/results/metrics_summary.csv


In [11]:
# Training args, embeddings + tiny LoRA
args = TrainingArguments(
    output_dir="/content/Wake2vec_adapter",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=6e-4,
    num_train_epochs=3,
    bf16=torch.cuda.is_available(),
    logging_steps=25,

    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    lr_scheduler_type="cosine",
    warmup_ratio=0.10,                   # ~10% warmup
    max_grad_norm=1.0,

    weight_decay=0.0,
    seed=42,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,

    report_to=[]
)

# Optional for speed
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    data_collator=collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model("/content/Wake2vec_adapter/final")
tok.save_pretrained("/content/Wake2vec_adapter/final_tok")
print("Saved adapter + tokenizer to /content/Wake2vec_adapter")


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,8.856500,8.768445
200,7.873900,7.821990
300,7.711000,7.724489


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the

Saved adapter + tokenizer to /content/Wake2vec_adapter


In [12]:
# tie output head to input embeddings (paranoia pass)
if model.get_output_embeddings() is not None:
    model.get_output_embeddings().weight = model.get_input_embeddings().weight
model.config.use_cache = True

In [16]:
import unicodedata, re
from sklearn.metrics.pairwise import cosine_similarity
LETTER = re.compile(r"^[A-Za-zÀ-ÖØ-öø-ÿ][A-Za-zÀ-ÖØ-öø-ÿ'’-]*$")

def is_latin(ch):
    try: return ("LATIN" in unicodedata.name(ch)) or ch in "'’-"
    except ValueError: return False

def is_wordlike(tok):
    if " " in tok: return False
    if not LETTER.match(tok): return False
    return all(is_latin(c) for c in tok)

vocab = tok.get_vocab(); inv = {i:t for t,i in vocab.items()}
allowed_ids = [i for i,t in inv.items() if is_wordlike(t) or t in new_terms or (" "+t) in new_terms]

W = model.get_input_embeddings().weight.detach().cpu().float().numpy() # Convert to float32 before numpy
Wa = W[allowed_ids] / (np.linalg.norm(W[allowed_ids], axis=1, keepdims=True) + 1e-12)

def clean_neighbors(terms, k=8):
    out = {}
    for t in terms[:30]:
        tid = tok.convert_tokens_to_ids(t)
        if tid == tok.unk_token_id: continue
        v = W[tid:tid+1] / (np.linalg.norm(W[tid:tid+1], axis=1, keepdims=True) + 1e-12)
        sims = (v @ Wa.T)[0]
        top = np.argpartition(-sims, range(min(k, len(sims))))[:k]
        top = top[np.argsort(-sims[top])]
        out[t] = [inv[allowed_ids[j]] for j in top]
    return out

probe = new_terms[:12]
nn = clean_neighbors(probe, k=8)
for k,v in nn.items():
    print(f"{k:>20} -> {v}")

         paùpulation -> ['paùpulation', 'Radius', 'whacked', 'fingerforce', 'loutgout', 'hrte', 'corsets', 'aff']
         générations -> ['générations', 'phassionable', 'aleguere', 'bilaws', 'danelagh', 'siss', 'messmother', 'wasteward']
           introdùce -> ['introdùce', 'madiens', 'plentifolks', 'skekels', 'headth', 'schayns', 'mutherer', 'legend']
           fainéants -> ['fainéants', 'üg', 'block', 'dreans', "ran's", 'Progress', 'basidens', 'nect']
           tricarême -> ['tricarême', 'whitecaps', 'dorass', 'onie', 'becket', 'heartsfoot', 'goodrid', 'peartree']
           deathfête -> ['deathfête', 'mesdaims', 'rawhoney', 'ungump', 'welled', 'pardonnez-leur', 'marsellas', 'chewable']
           grandmère -> ['grandmère', "temple's", 'openair', 'roundshows', 'simper', 'durrydunglecks', 'trancenania', 'documentclass']
           primamère -> ['primamère', 'tipside', 'chrysolite', 'listeneath', 'bounds', 'twosingwoolow', 'flageolettes', 'ursulinka']
            brofèsor -> ['brof

In [22]:
from transformers import LogitsProcessorList, RepetitionPenaltyLogitsProcessor, NoRepeatNGramLogitsProcessor

def complete_calm(prompt, max_new_tokens=80):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    processors = LogitsProcessorList([
        RepetitionPenaltyLogitsProcessor(penalty=1.18),
        NoRepeatNGramLogitsProcessor(4),
    ])
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.95,
            top_p=0.90,
            top_k=50,
            typical_p=0.95,
            logits_processor=processors,
            renormalize_logits=True,
            cache_implementation="static"
        )
    return tok.decode(out[0], skip_special_tokens=True)


In [ ]:
# Smoothing pass: embeddings-only, short & gentle
for p in model.parameters(): p.requires_grad = False
model.get_input_embeddings().weight.requires_grad_(True)

from transformers import TrainingArguments, Trainer
args2 = TrainingArguments(
    output_dir="/content/Wake2vec_adapter_smooth",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,      # lower LR
    num_train_epochs=0.7,    # < 1 epoch
    bf16=torch.cuda.is_available(), # Changed from fp16=True
    logging_steps=25,
    evaluation_strategy="no",
    report_to=[]
)
Trainer(model=model, args=args2, train_dataset=ds["train"], data_collator=collator).train()

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
25,6.989400
50,6.618200


In [24]:
from transformers import (
    LogitsProcessor, LogitsProcessorList,
    RepetitionPenaltyLogitsProcessor,
    NoRepeatNGramLogitsProcessor
)
import torch, math

# tiny bias helper: nudge Wake tokens, common stopwords down
class LogitBiasProcessor(LogitsProcessor):
    def __init__(self, bias_ids, bias_values):
        self.bias = torch.zeros(max(bias_ids)+1, dtype=torch.float32, device=bias_values.device)
        self.bias[:]=0
        self.bias[bias_ids] = bias_values

    def __call__(self, input_ids, scores):
        # scores: [batch, vocab]
        return scores + self.bias

def build_decoder_processors(tok, new_terms, wake_boost=0.35, stop_suppress=0.25):
    # Wake lexicon (+ ▁variants)
    wake_tokens = set()
    for t in new_terms[:1000]:
        for form in (t, "▁"+t if not t.startswith("▁") else t):
            tid = tok.convert_tokens_to_ids(form)
            if tid != tok.unk_token_id:
                wake_tokens.add(tid)

    # A tiny English stopword set
    stopish = """
    the of and to a in that it is you i me we he she they my your his her our their for on as with was be are
    this at from or an so if but by have has had not just do did does can will would could should
    """.split()
    stop_ids = [tok.convert_tokens_to_ids(w) for w in stopish if tok.convert_tokens_to_ids(w) != tok.unk_token_id]

    # Bias vector (same length as vocab, added to logits each step)
    vocab_size = len(tok)
    bias = torch.zeros(vocab_size)
    # +wake_boost to Wake tokens, -stop_suppress to very common function words
    for wid in wake_tokens: bias[wid] += wake_boost
    for sid in stop_ids:    bias[sid]  -= stop_suppress

    processors = LogitsProcessorList([
        RepetitionPenaltyLogitsProcessor(1.20),
        NoRepeatNGramLogitsProcessor(4),
        # plug a LogitBiasProcessor at call-time (needs device)
    ])
    return processors, bias

processors_base, bias_vec = build_decoder_processors(tok, new_terms, wake_boost=0.35, stop_suppress=0.25)

def complete_portfolio(prompt, max_new_tokens=90):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    # move bias to device and wrap as processor
    bias_proc = LogitBiasProcessor(
        bias_ids=torch.arange(len(bias_vec), device=model.device),
        bias_values=bias_vec.to(model.device),
    )
    processors = LogitsProcessorList(list(processors_base) + [bias_proc])

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.90,      # cooler
            top_p=0.88,
            top_k=40,
            typical_p=0.95,
            logits_processor=processors,
            renormalize_logits=True,
            no_repeat_ngram_size=4,
            cache_implementation="static",
        )
    return tok.decode(out[0], skip_special_tokens=True)

In [25]:
def complete(prompt, max_new_tokens=60):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=1.15,
            top_p=0.95,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3
        )
    return tok.decode(out[0], skip_special_tokens=True)

tests = [
    "By the river I thought of — Earwicker, the prankquean, and",
    "At night I dream of",
    "In the story of this book,",
    "Explain gradient descent in the style of Joyce: riverrun,"
]
for p in tests:
    print("\n===", p, "\n", complete(p))


=== By the river I thought of — Earwicker, the prankquean, and 
 By the river I thought of — Earwicker, theprankquean, and anfirsk. —paudraic's of the one if be he, at a you? —To thefirsh you on not on was would me at her m-overline is for beer girchipperchapper, to his own, he the these or all up as my pums! Not we have no

=== At night I dream of 
 At night I dream of the go, he he for a mimam to was the kot. But to this I it and there his samer and not the pagant with the raun, she if an cyi, in the rore chainganger's. Thas. Pieleo in

=== In the story of this book, 
 In the story of this book, in so me his gimsorer was afirse? [124] Gan with Oymly. —Neistishie, them and the Lunut, all a care, your bill in his kharin of to old on his fuck of

=== Explain gradient descent in the style of Joyce: riverrun, 
 Explain gradient descent in the style of Joyce:riverrun, bante and O of her all is him of a beo for himed her have a man m-overline or in the raush to are a ao, to to boom and you

In [ ]:
# Recompute metrics and append (keeps history)
import numpy as np, pandas as pd, json, os
os.makedirs("results", exist_ok=True)

# If you snapped a baseline BEFORE training, reuse it; otherwise just log "after" once.
# Expect: PIP > 0 (some drift), isotropy reasonable (~0.2–0.9 depending on base), overlap < 1.0.
W = model.get_input_embeddings().weight.detach().cpu().numpy()

# Probe terms = mix of new tokens + anchors
anchors = ["river","history","book","dream","night","language","irish","dublin","cork"]
probe = [t for t in new_terms[:150] if tok.convert_tokens_to_ids(t) != tok.unk_token_id] + anchors
ids = [tok.convert_tokens_to_ids(t) for t in probe]
P = W[ids] @ W[ids].T

# If you have a saved baseline P0.npy + probe_ids.npy, load and compute PIP
pip = None
try:
    import glob
    base_dir = sorted(glob.glob("/content/Wake2vec_baseline_*"))[-1]
    base_ids = np.load(base_dir + "/probe_ids.npy")
    P0 = np.load(base_dir + "/P0.npy")
    # align if probe changed size/order
    if np.array_equal(base_ids, np.array(ids)):
        pip = float(np.linalg.norm(P0 - P, 'fro') / (np.linalg.norm(P0, 'fro') + 1e-12))
except Exception as e:
    pip = None

# Isotropy
X = W[ids] - W[ids].mean(0, keepdims=True)
C = (X.T @ X) / max(1, (len(ids)-1))
evals = np.clip(np.linalg.eigvalsh(C), 1e-12, None)
isotropy = float(np.exp(np.log(evals).mean()) / evals.mean())

# Top-10 overlap (if baseline available)
topk = 10
top_now = np.argsort(-P, axis=1)[:, :topk]
overlap = None
if pip is not None:
    top_base = np.argsort(-P0, axis=1)[:, :topk]
    overlap = float(np.mean([len(set(top_base[i]).intersection(set(top_now[i]))) / topk for i in range(len(ids))]))

row = {"pip_loss": pip if pip is not None else np.nan,
       "isotropy": isotropy,
       "top10_overlap": overlap if overlap is not None else np.nan,
       "num_probe_tokens": len(ids)}
df_path = "results/metrics_summary.csv"
pd.DataFrame([row]).to_csv(df_path, mode=("a" if os.path.exists(df_path) else "w"),
                           header=not os.path.exists(df_path), index=False)
print("Appended metrics to", df_path)

# Neighbor table (cleaned) for README appendix
import unicodedata, re
LETTER = re.compile(r"^[A-Za-zÀ-ÖØ-öø-ÿ][A-Za-zÀ-ÖØ-öø-ÿ'’-]*$")
def is_latin(ch):
    try: return ("LATIN" in unicodedata.name(ch)) or ch in "'’-"
    except ValueError: return False
def is_wordlike(tok):
    return ("▁" not in tok) and LETTER.match(tok) and all(is_latin(c) for c in tok)

vocab = tok.get_vocab(); inv = {i:t for t,i in vocab.items()}
allowed_ids = [i for i,t in inv.items() if is_wordlike(t) or t in new_terms or ("▁"+t) in new_terms]
Wa = W[allowed_ids] / (np.linalg.norm(W[allowed_ids], axis=1, keepdims=True) + 1e-12)

from sklearn.metrics.pairwise import cosine_similarity
def clean_neighbors_one(idx, k=8):
    v = W[idx:idx+1] / (np.linalg.norm(W[idx:idx+1], axis=1, keepdims=True) + 1e-12)
    sims = (v @ Wa.T)[0]
    top = np.argpartition(-sims, range(min(k, len(sims))))[:k]
    top = top[np.argsort(-sims[top])]
    return [inv[allowed_ids[j]] for j in top]

nn_after = {}
for t in probe[:20]:
    tid = tok.convert_tokens_to_ids(t)
    if tid != tok.unk_token_id:
        nn_after[t] = clean_neighbors_one(tid, k=8)
json.dump(nn_after, open("results/neighbors_after.json","w"), ensure_ascii=False, indent=2)
print("Wrote results/neighbors_after.json")

In [ ]:
# hero fig
import umap, matplotlib.pyplot as plt, numpy as np, textwrap, os, random
os.makedirs("results", exist_ok=True)

probe_show = list(dict.fromkeys(new_terms[:40] + ["river","history","book","dream","night","language","irish","dublin"]))
probe_ids = [tok.convert_tokens_to_ids(t) for t in probe_show if tok.convert_tokens_to_ids(t) != tok.unk_token_id]
vecs = model.get_input_embeddings().weight[probe_ids].detach().cpu().numpy()

mapper = umap.UMAP(n_neighbors=8, min_dist=0.1, metric="cosine", random_state=42)
xy = mapper.fit_transform(vecs)

# three calmer samples
samples = {
  "By the river I thought of — Earwicker, the prankquean, and": complete_portfolio("By the river I thought of — Earwicker, the prankquean, and", 120),
  "In the story of this book, HCE remembers that": complete_portfolio("In the story of this book, HCE remembers that", 120),
  "Explain gradient descent in the style of Joyce: riverrun,": complete_portfolio("Explain gradient descent in the style of Joyce: riverrun,", 120),
}

plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.scatter(xy[:,0], xy[:,1])
inv = {i:t for t,i in tok.get_vocab().items()}
for pid, (x,y) in zip(probe_ids, xy):
    plt.text(x, y, inv.get(int(pid), "?"), fontsize=8)
plt.title("Wake2vec: neighborhood map (UMAP)")

plt.subplot(1,2,2)
y0=1.0
for k,v in samples.items():
    txt = k + "\n" + textwrap.fill(v.replace("\n"," "), width=44)
    plt.text(0.0, y0, txt, fontsize=8, va='top')
    y0 -= 0.32
plt.axis('off'); plt.title("Sample completions")
plt.tight_layout()
plt.savefig("results/hero.png", dpi=200)
print("Saved results/hero.png")


In [21]:
# Export minimal adapter pack: changed embedding rows (bare +   forms)
save_dir = Path("/content/Wake2vec_adapter/minipack"); save_dir.mkdir(parents=True, exist_ok=True)
changed = [tok.convert_tokens_to_ids(t) for t in (new_terms + [" "+t for t in new_terms if not t.startswith(" ")])]
changed = [i for i in changed if i != tok.unk_token_id]
emb_slice = model.get_input_embeddings().weight[changed].detach().cpu().float().numpy() # Convert to float32 before numpy
np.save(save_dir / "new_token_ids.npy", np.array(changed, dtype=np.int32))
np.save(save_dir / "new_token_vectors.npy", emb_slice)
(Path(save_dir / "README.txt").write_text(
    "Rows from input embedding for Wake2vec tokens (bare +  forms).\n"
    "Apply to base model by index assignment.\n", encoding="utf-8"))
print("Mini pack saved:", save_dir)

Mini pack saved: /content/Wake2vec_adapter/minipack
